In [9]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import re
import math

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

plt.style.use('seaborn-whitegrid')
matplotlib.rcParams['figure.figsize'] = (10.0, 4.0)

# Base Tank Class
This class is the tamplate that will hold the genetic code as its instruction set that can be run.
Movement is based on being placed in a 10 x 10 grid and can be modified by the GRID_SIZE var.

In [28]:
GRID_SIZE = 10

class Tank :    
    def __init__(self, X, Y, facing, instructions) : #string of instructions to be executed every turn of the evaluation sim
        self.X = X #starting x location
        self.Y = Y #starting y location
        self.facing = facing #starting facing direction in degrees (E: 0, N: 90, W: 180, S: 270)
        
        self.shields = False #if shields are powered on or not
        
        self.instructions = instructions
        self.instructCode = compile(instructions, 'mulstring', 'exec') #compile the instructions to be run with exec()
        
        self.energy = 1000 #start with 1000 energy
        self.roundEnergy = 0 #the energy lost in a single round. This is subtracted at end of round
        
        
    def execInstructions(self) :
        exec(self.instructCode) #run this tanks logic code
        
        self.TotalEnergyLost() #calculate all the energy lost after running these instructions
        self.shields = False #turn shields off
    
    #set up base functions for the tank
    def Forw(self) : #move tank forward one step in the facing direction
        self.roundEnergy += 10 #moving costs 10 energy
        
        if self.facing == 0 and self.X != GRID_SIZE : #facing right, moving in X direction
            self.X += 1 #increase x position
        elif self.facing == 90 and self.Y != 0 : #facing up, moving in Y direction
            self.Y -= 1 #decrease y position
        elif self.facing == 180 and self.X != 0 : #facing left, moving in X direction
            self.X -= 1 #decrease x position
        elif self.facing == 270 and self.Y != GRID_SIZE : #facing down, moving in Y direction
            self.Y += 1 #increase y position
        
        print("Forw")
        print("   Coord: (" + str(self.X) + "," + str(self.Y) + ")")
    
    def TurnL(self) : #turn tank left
        self.roundEnergy += 10 #turning costs 10 energy
        
        self.facing = (self.facing + 90) % 360 #rotate to next cardinal directions and mod to reset to 0 if 360
        
        print("TurnL")
        print("   Dir: " + str(self.facing))
        
    def TurnR(self) : #turn tank right
        self.roundEnergy += 10 #turning costs 10 energy
        
        self.facing = self.facing - 90 #rotate to next cardinal directions
        self.facing -= 360 * math.floor(self.facing * (1. / 360.)) #standardize [0, 360]
        
        print("TurnR")
        print("   Dir: " + str(self.facing))
        
    def ActivateShields(self) : #activate tank's shields
        self.roundEnergy += 20 #turning costs 10 energy
        
        self.shields = True #turn on shields
        
        print("Shields")
        print("   On")
        
    def Wait(self) : #no-op
        pass #costs no energy
        
    def TotalEnergyLost(self) : #apply all lost energy at end of round
        self.energy -= self.roundEnergy #subtract all energy lost in this round
        self.roundEnergy = 0 #reset energy for the next round
    
    #toString Function for printing
    def __str__(self) :
        return ("Coord: (" + str(self.X) + "," + str(self.Y) +
                ")\nDir: " + str(self.facing) +
                "\nEnergy: " + str(self.energy))
        
        
#Test Tank class code
tank = Tank(2, 2, 90, "self.Forw() \nself.ActivateShields()")
tank.execInstructions()
tank.TurnR()
tank.TurnL()

print(tank)

Forw
   Coord: (2,1)
Shields
   On
TurnR
   Dir: 0
TurnL
   Dir: 90
Coord: (2,1)
Dir: 90
Energy: 970


# Deap GP Section
https://deap.readthedocs.io/en/master/tutorials/advanced/gp.html
Used for creating the genetic tank variants

In [35]:
def if_wall(tank, output1, output2) : #execute output1 if tank is directly facing a wall, otherwise output2
    #             Facing right wall                             Facing top wall                        Facing left wall                        Facing bottom wall
    facingWall = (tank.X == GRID_SIZE and tank.facing == 0) or (tank.Y == 0 and tank.facing == 90) or (tank.X == 0 and tank.facing == 180) or (tank.Y == GRID_SIZE and tank.facing == 270)
    return output1 if facingWall else output2



#pset = PrimitiveSetTyped("main", [bool, float], float)
#pset.addPrimitive(if_wall, [Tank, float, float], float)

tank = Tank(10, 10, 270, "")
print(if_wall(tank, "True", "False"))
tank.TurnL()
print(if_wall(tank, "True", "False"))
tank.TurnL()
print(if_wall(tank, "True", "False"))
tank.TurnL()
print(if_wall(tank, "True", "False"))


True
TurnL
   Dir: 0
True
TurnL
   Dir: 90
False
TurnL
   Dir: 180
False
